<a href="https://colab.research.google.com/github/Ravikula/power_systems/blob/main/Power_Systems_Assignment_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cmath
import math
import numpy as np

In [2]:
#Data

pg = 100        #MVA    Generator rated power
pm = 100        #MVA    Motor rated power
vg = 13.8       #kV     Generator rated voltage
vm = 13.8       #kV     Motor rated voltage

vp_tf12 = 13.8  #kV     Transformer 1,2 primary voltage
vs_ft12 = 132   #kV     Transformer 1,2 secondary voltage

vp_tf3 = 13.8   #kV     Transformer 3 primary voltage
vs_ft3 = 132    #kV     Transformer 3 secondary voltage

xgg = 0.04j      #pu     Generator 1,2 Grounding impedance
xgm = 0.05j      #pu     Motor grounding impedance


#POSITIVE SEQUENCE

pos_seq = {"xg" : 0.15j,    #0 - generator positive sequance reactance
           "xm" : 0.2j,     #1 - motor positive sequance reactance
           "xt" : 0.105j,   #2 - Transmission line positive sequance reactance
           "xtf12": 0.10j,  #3 - Transformer 1,2 positive sequance reactance
           "xtf3":0.10j}    #4 - transformer 3 positive sequance reactance

#NEGATIVE SEQUENCE

neg_seq = {"xg" : 0.170j,   #0 - generator negative sequance reactance
           "xm" : 0.21j,    #1 - motor negative sequance reactance
           "xt" : 0.105j,   #2 - Transmission line negative sequance reactance
           "xtf12": 0.10j,  #3 - Transformer 1,2 negative sequance reactance
           "xtf3":0.10j}    #4 - transformer 3 negative sequance reactance


#ZERO SEQUANCE

zero_seq = {"xg" : 0.050j,  #0 - generator negative sequance reactance
           "xm" : 0.1j,     #1 - motor negative sequance reactance
           "xt" : 0.315j,   #2 - Transmission line negative sequance reactance
           "xtf12": 0.100j, #3 - Transformer 1,2 negative sequance reactance
           "xtf3":0.10j}    #4 - transformer 3 negative sequance reactance

a = cmath.rect(1, math.radians(120))
a2 = cmath.rect(1, math.radians(240))

#TRANSFORMATION MATRIX
T = np.array([[1, 1, 1],
              [1, a2, a],
              [1, a, a2]])

#INVERSE TRANSFORMATION MATRIX
T_1 = np.array([[1, 1, 1],
                [1, a, a2],
                [1, a2, a]])

d = 4     #No of decimal places

In [3]:
#Function to calculate parallel impedance

def z_p(a,b):
  c = 1/((1/a)+(1/b))
  return c

In [4]:
#PART 1

#POSITIVE SEQUENCE

xp1 = pos_seq["xm"]+pos_seq["xtf3"]+pos_seq["xt"]
xp2 = z_p(pos_seq["xtf12"]+pos_seq["xg"],pos_seq["xtf12"]+pos_seq["xg"])
Z_th_pos = z_p(xp1, xp2)

print("Z_th_pos: "+str(round(abs(Z_th_pos),d)) + " (pu)  " + str(math.degrees(cmath.phase(Z_th_pos)))+" (degrees)")


#NEGATIVE SEQUENCE

xn1 = neg_seq["xm"]+neg_seq["xtf3"]+neg_seq["xt"]
xn2 = z_p(neg_seq["xtf12"]+neg_seq["xg"],neg_seq["xtf12"]+neg_seq["xg"])
Z_th_neg = z_p(xn1, xn2)

print("Z_th_neg: "+str(round(abs(Z_th_neg),d)) + " (pu)  " + str(math.degrees(cmath.phase(Z_th_neg)))+" (degrees)")


#ZERO SEQUENCE

Z_th_zero = zero_seq["xtf12"]+zero_seq["xg"]+xgg*3
print("Z_th_zero: "+str(round(abs(Z_th_zero),d)) + " (pu)  " + str(math.degrees(cmath.phase(Z_th_zero)))+" (degrees)")

Z_th_pos: 0.0955 (pu)  90.0 (degrees)
Z_th_neg: 0.1019 (pu)  90.0 (degrees)
Z_th_zero: 0.27 (pu)  90.0 (degrees)


In [5]:
#PART 2

# For balanced faults, only the positive sequance is considered.
vf_pre = 1.02 +0j

I0_2 = 0
Ip_2 = vf_pre/Z_th_pos
In_2 = 0

#Sequence currents
I_seq_2 = np.array([[I0_2],
                  [Ip_2],
                  [In_2]])

#Fault currents
If_ph_2 = np.dot(T, I_seq_2)

print("Ia = : " +str(round(np.abs(If_ph_2[0,0]),d)) + " (pu) " +str(round(np.angle(If_ph_2[0,0], deg=True),d)) + " (degree)")
print("Ib = : " +str(round(np.abs(If_ph_2[1,0]),d)) + " (pu) " +str(round(np.angle(If_ph_2[1,0], deg=True),d)) + " (degree)")
print("Ic = : " +str(round(np.abs(If_ph_2[2,0]),d)) + " (pu) " +str(round(np.angle(If_ph_2[2,0], deg=True),d)) + " (degree)")


Ia = : 10.6785 (pu) -90.0 (degree)
Ib = : 10.6785 (pu) 150.0 (degree)
Ic = : 10.6785 (pu) 30.0 (degree)


In [6]:

#PART 3

#Assume fault occur between phase A and Ground. Therefore, current flows only in phase A. Therefore, Ib=0 and Ic =0
Ip_3 = vf_pre/(Z_th_pos+Z_th_neg+Z_th_zero)
In_3 = Ip_3
I0_3 = Ip_3

#Using matrix
I_seq_3 = np.array([[I0_3],
                    [Ip_3],
                    [In_3]])

If_ph_3 = np.dot(T, I_seq_3)

#Using general formulas
Ia = Ip_3 + In_3 + I0_3
Ib =0
Ic =0

print("Ia = : " +str(round(abs(Ia),d)) + "  " +str(round(np.angle(Ia, deg=True),d)))
print("Ib = : " +str(abs(Ib)) + "  " +str(np.angle(Ib, deg=True)))
print("Ic = : " +str(abs(Ic)) + "  " +str(np.angle(Ic, deg=True)))

print("................")

print("Ia = : " +str(round(np.abs(If_ph_3[0,0]),d)) + " (pu) " +str(round(np.angle(If_ph_3[0,0], deg=True),d)) + " (degree)")
print("Ib = : " +str(round(np.abs(If_ph_3[1,0]),d)) + " (pu) " +str(round(np.angle(If_ph_3[1,0], deg=True),d)) + " (degree)")
print("Ic = : " +str(round(np.abs(If_ph_3[2,0]),d)) + " (pu) " +str(round(np.angle(If_ph_3[2,0], deg=True),d)) + " (degree)")


Ia = : 6.5471  -90.0
Ib = : 0  0.0
Ic = : 0  0.0
................
Ia = : 6.5471 (pu) -90.0 (degree)
Ib = : 0.0 (pu) 26.5651 (degree)
Ic = : 0.0 (pu) 26.5651 (degree)


In [7]:
#PART 4

Ip_4 = vf_pre/(Z_th_pos+Z_th_neg)

In_4 = -Ip_4
I0_4 = 0

I_seq_4 = np.array([[I0_4],
                    [Ip_4],
                    [In_4]])

If_ph_4 = np.dot(T, I_seq_4)

print("Ia = : " +str(round(np.abs(If_ph_4[0,0]),d)) + " (pu) " +str(round(np.angle(If_ph_4[0,0], deg=True),d)) + " (degree)")
print("Ib = : " +str(round(np.abs(If_ph_4[1,0]),d)) + " (pu) " +str(round(np.angle(If_ph_4[1,0], deg=True),d)) + " (degree)")
print("Ic = : " +str(round(np.abs(If_ph_4[2,0]),d)) + " (pu) " +str(round(np.angle(If_ph_4[2,0], deg=True),d)) + " (degree)")

Ia = : 0.0 (pu) 0.0 (degree)
Ib = : 8.9506 (pu) 180.0 (degree)
Ic = : 8.9506 (pu) -0.0 (degree)


In [8]:
#PART 5

#Assume fault is between phase B and C and ground. Then VB = VC = 0, IA = 0 and IB+IC = IF, Vp = Vn = V0

Ip_5 = vf_pre/(Z_th_pos + z_p(Z_th_neg,Z_th_zero))
Vp = z_p(Z_th_neg,Z_th_zero)*Ip_5

In_5 = -(Vp/Z_th_neg)
I0_5 = -(Vp/Z_th_zero)

I_seq_5 = np.array([[I0_5],
                    [Ip_5],
                    [In_5]])
If_ph_5 = np.dot(T, I_seq_5)


print("Ia = : " +str(round(np.abs(If_ph_5[0,0]),d)) + " (pu) " +str(round(np.angle(If_ph_5[0,0], deg=True),d)) + " (degree)")
print("Ib = : " +str(round(np.abs(If_ph_5[1,0]),d)) + " (pu) " +str(round(np.angle(If_ph_5[1,0], deg=True),d)) + " (degree)")
print("Ic = : " +str(round(np.abs(If_ph_5[2,0]),d)) + " (pu) " +str(round(np.angle(If_ph_5[2,0], deg=True),d)) + " (degree)")


Ia = : 0.0 (pu) 0.0 (degree)
Ib = : 9.3302 (pu) 164.6303 (degree)
Ic = : 9.3302 (pu) 15.3697 (degree)


In [9]:
#PART 6 - Normal method

vp_l_g = vf_pre-Z_th_pos*(Ip_3)
vn_l_g = -Z_th_neg*(Ip_3)
v0_l_g = -Z_th_zero*(Ip_3)

V_seq_l_g = np.array([[v0_l_g],
                      [vp_l_g],
                      [vn_l_g]])

Vf_ph_l_g = np.dot(T, V_seq_l_g)


print("Va = : " +str(round(np.abs(Vf_ph_l_g[0,0]),d)) + " (pu) " +str(round(np.angle(Vf_ph_l_g[0,0], deg=True),d)) + " (degree)")
print("Vb = : " +str(round(np.abs(Vf_ph_l_g[1,0]),d)) + " (pu) " +str(round(np.angle(Vf_ph_l_g[1,0], deg=True),d)) + " (degree)")
print("Vc = : " +str(round(np.abs(Vf_ph_l_g[2,0]),d)) + " (pu) " +str(round(np.angle(Vf_ph_l_g[2,0], deg=True),d)) + " (degree)")


Va = : 0.0 (pu) 0.0 (degree)
Vb = : 1.2581 (pu) -134.6303 (degree)
Vc = : 1.2581 (pu) 134.6303 (degree)


In [10]:
#PART 6 - Matrix Method

VF = np.array([[0],[vf_pre],[0]])

Z = np.array([[Z_th_zero, 0, 0],
              [0, Z_th_pos, 0],
              [0, 0, Z_th_neg]])

I_seq_6 = np.array([[I0_3],[Ip_3],[In_3]])

v_seq_6 = VF - np.dot(Z,I_seq_6)

Vf_ph_l_g2 = np.dot(T,v_seq_6)


print("Va = : " +str(round(np.abs(Vf_ph_l_g2[0,0]),d)) + " (pu) " +str(round(np.angle(Vf_ph_l_g2[0,0], deg=True),d)) + " (degree)")
print("Vb = : " +str(round(np.abs(Vf_ph_l_g2[1,0]),d)) + " (pu) " +str(round(np.angle(Vf_ph_l_g2[1,0], deg=True),d)) + " (degree)")
print("Vc = : " +str(round(np.abs(Vf_ph_l_g2[2,0]),d)) + " (pu) " +str(round(np.angle(Vf_ph_l_g2[2,0], deg=True),d)) + " (degree)")


Va = : 0.0 (pu) 0.0 (degree)
Vb = : 1.2581 (pu) -134.6303 (degree)
Vc = : 1.2581 (pu) 134.6303 (degree)


In [11]:
#PART 7
Z = np.array([[Z_th_zero, 0, 0],
              [0, Z_th_pos, 0],
              [0, 0, Z_th_neg]])

I_seq_7 = np.array([[I0_4],[Ip_4],[In_4]])

v_seq_7 = VF - np.dot(Z,I_seq_7)

Vf_ph_l_l = np.dot(T,v_seq_7)

print("Va = : " +str(round(np.abs(Vf_ph_l_l[0,0]),d)) + " (pu) " +str(round(np.angle(Vf_ph_l_l[0,0], deg=True),d)) + " (degree)")
print("Vb = : " +str(round(np.abs(Vf_ph_l_l[1,0]),d)) + " (pu) " +str(round(np.angle(Vf_ph_l_l[1,0], deg=True),d)) + " (degree)")
print("Vc = : " +str(round(np.abs(Vf_ph_l_l[2,0]),d)) + " (pu) " +str(round(np.angle(Vf_ph_l_l[2,0], deg=True),d)) + " (degree)")

Va = : 1.0528 (pu) 0.0 (degree)
Vb = : 0.5264 (pu) 180.0 (degree)
Vc = : 0.5264 (pu) 180.0 (degree)


In [12]:
#PART 8
#!!!!!!!! Vb and Vc angles should be zero
Z = np.array([[Z_th_zero, 0, 0],
              [0, Z_th_pos, 0],
              [0, 0, Z_th_neg]])

I_seq_8 = np.array([[I0_5],[Ip_5],[In_5]])

v_seq_8 = VF - np.dot(Z,I_seq_8)

Vf_ph_l_l_g = np.dot(T,v_seq_8)

print("Va = : " +str(round(np.abs(Vf_ph_l_l_g[0,0]),d)) + " (pu) " +str(round(np.angle(Vf_ph_l_l_g[0,0], deg=True),d)) + " (degree)")
print("Vb = : " +str(round(np.abs(Vf_ph_l_l_g[1,0]),d)) + " (pu) " +str(round(np.angle(Vf_ph_l_l_g[1,0], deg=True),d)) + " (degree)")
print("Vc = : " +str(round(np.abs(Vf_ph_l_l_g[2,0]),d)) + " (pu) " +str(round(np.angle(Vf_ph_l_l_g[2,0], deg=True),d)) + " (degree)")

Va = : 1.3354 (pu) 0.0 (degree)
Vb = : 0.0 (pu) 116.5651 (degree)
Vc = : 0.0 (pu) 116.5651 (degree)


In [13]:
#PART 9

Im = Ip_2*(xp2)/(xp1+xp2)

vd_f = vf_pre-Im*(pos_seq["xm"]+pos_seq["xtf3"])

vd_seq = np.array([[0],[vd_f],[0]])

vd_ph = np.dot(T,vd_seq)

print("Va = : " +str(round(np.abs(vd_ph[0,0]),d)) + " (pu) " +str(round(np.angle(vd_ph[0,0], deg=True),d)) + " (degree)")
print("Vb = : " +str(round(np.abs(vd_ph[1,0]),d)) + " (pu) " +str(round(np.angle(vd_ph[1,0], deg=True),d)) + " (degree)")
print("Vc = : " +str(round(np.abs(vd_ph[2,0]),d)) + " (pu) " +str(round(np.angle(vd_ph[2,0], deg=True),d)) + " (degree)")

Va = : 0.2644 (pu) 0.0 (degree)
Vb = : 0.2644 (pu) -120.0 (degree)
Vc = : 0.2644 (pu) 120.0 (degree)


In [14]:
#PART 10

v0_10 = v_seq_6[0,0]
vp_10 = v_seq_6[1,0]
vn_10 = v_seq_6[2,0]

Ig_p_10 = (vf_pre-vp_10)/(pos_seq["xm"]+pos_seq["xtf3"]+pos_seq["xt"])

vd_p_10 = vf_pre - Ig_p_10*(pos_seq["xm"]+pos_seq["xtf3"])

vd_n_10 = vn_10*((neg_seq["xm"]+neg_seq["xtf3"])/(neg_seq["xm"]+neg_seq["xtf3"]+neg_seq["xt"]))

vd_0_10 = v0_10

vd_seq_10 = np.array([[vd_0_10],[vd_p_10],[vd_n_10]])

vd_10 = np.dot(T,vd_seq_10)

print("Va = : " +str(round(np.abs(vd_10[0,0]),d)) + " (pu) " +str(round(np.angle(vd_10[0,0], deg=True),d)) + " (degree)")
print("Vb = : " +str(round(np.abs(vd_10[1,0]),d)) + " (pu) " +str(round(np.angle(vd_10[1,0], deg=True),d)) + " (degree)")
print("Vc = : " +str(round(np.abs(vd_10[2,0]),d)) + " (pu) " +str(round(np.angle(vd_10[2,0], deg=True),d)) + " (degree)")


Va = : 0.1103 (pu) 0.0 (degree)
Vb = : 1.2961 (pu) -136.4246 (degree)
Vc = : 1.2961 (pu) 136.4246 (degree)


In [15]:
#PART 11

v0_11 = v_seq_7[0,0]
vp_11 = v_seq_7[1,0]
vn_11 = v_seq_7[2,0]

Ig_p_11 = (vf_pre-vp_11)/(pos_seq["xm"]+pos_seq["xtf3"]+pos_seq["xt"])

vd_p_11 = vf_pre - Ig_p_11*(pos_seq["xm"]+pos_seq["xtf3"])

vd_n_11 = vn_11*((neg_seq["xm"]+neg_seq["xtf3"])/(neg_seq["xm"]+neg_seq["xtf3"]+neg_seq["xt"]))

vd_0_11 = v0_11

vd_seq_11 = np.array([[vd_0_11],[vd_p_11],[vd_n_11]])

vd_11 = np.dot(T,vd_seq_11)

print("Va = : " +str(round(np.abs(vd_11[0,0]),d)) + " (pu) " +str(round(np.angle(vd_11[0,0], deg=True),d)) + " (degree)")
print("Vb = : " +str(round(np.abs(vd_11[1,0]),d)) + " (pu) " +str(round(np.angle(vd_11[1,0], deg=True),d)) + " (degree)")
print("Vc = : " +str(round(np.abs(vd_11[2,0]),d)) + " (pu) " +str(round(np.angle(vd_11[2,0], deg=True),d)) + " (degree)")

Va = : 1.0476 (pu) 0.0 (degree)
Vb = : 0.5705 (pu) -156.6457 (degree)
Vc = : 0.5705 (pu) 156.6457 (degree)


In [16]:
#PART 12

v0_12 = v_seq_8[0,0]
vp_12 = v_seq_8[1,0]
vn_12 = v_seq_8[2,0]

Ig_p_12 = (vf_pre-vp_12)/(pos_seq["xm"]+pos_seq["xtf3"]+pos_seq["xt"])

vd_p_12 = vf_pre - Ig_p_12*(pos_seq["xm"]+pos_seq["xtf3"])

vd_n_12 = vn_12*((neg_seq["xm"]+neg_seq["xtf3"])/(neg_seq["xm"]+neg_seq["xtf3"]+neg_seq["xt"]))

vd_0_12 = v0_12

vd_seq_12 = np.array([[vd_0_12],[vd_p_12],[vd_n_12]])

vd_12 = np.dot(T,vd_seq_12)

print("Va = : " +str(round(np.abs(vd_12[0,0]),d)) + " (pu) " +str(round(np.angle(vd_12[0,0], deg=True),d)) + " (degree)")
print("Vb = : " +str(round(np.abs(vd_12[1,0]),d)) + " (pu) " +str(round(np.angle(vd_12[1,0], deg=True),d)) + " (degree)")
print("Vc = : " +str(round(np.abs(vd_12[2,0]),d)) + " (pu) " +str(round(np.angle(vd_12[2,0], deg=True),d)) + " (degree)")

Va = : 1.3718 (pu) 0.0 (degree)
Vb = : 0.2273 (pu) -94.5943 (degree)
Vc = : 0.2273 (pu) 94.5943 (degree)
